<a href="https://colab.research.google.com/github/iodoform/HandFontMaker/blob/main/HandFontMaker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 手書きフォントメーカー
## 概要
タブレット等で手書きフォントが作成できるプログラム。
## 使い方
1. 下にあるフォント範囲選択で、フォントを作成したい文字範囲のチェックボックスを選択してください。複数選択可能。
1. メニューから`ランタイム＞すべてのセルを実行` を選択。
1. Google Drive のアクセスを求められるので、許可してください。
1. しばらく待つと、一番下にフォントを書き込みUIが表示されるので、「現在の文字」と書かれている文字を手書き入力画面に書き込み、`次の文字` ボタンを押してください。これを選択文字範囲で繰り返します。
1. 選択した文字範囲がすべて書き終わると「最後の文字です。フォント作成ボタンを押してください。」と表示されるので、指示に従い`フォント作成`ボタンを押してください。ちなみに、最後まで書かずとも、`フォント作成`ボタンを押すとその時点までのフォントファイルができます。
1. 出来上がったフォントは`/content/drive/MyDrive/fontdir/handfont.otf`からダウンロードできます。

## 備考
- 一度フォントを作成した後も、`/content/drive/MyDrive/fontdir/svgdir`にはフォントに変換する前のsvgデータが残っているので、再度このプログラムを実行すれば追加のフォントを作成することができます。<br>（使用例：半角英数字だけ作ってあとから全角ひらがなを追加）
- このプログラムの作者([iodoform](https://twitter.com/_iodoform_))は、他の人がこのプログラムを用いて作成したフォントに対して一切の権利を主張しません。

## 謝辞
構成コードは以下の方々のライブラリ、スニペット、コードを参考、あるいは依拠しています。 またnotebookは下記コード群のライセンスを継承します。
- [handfontgen: Handwriting Font Generator](https://github.com/nixeneko/handfontgen.git)
- [font-jp-subset](https://github.com/blivesta/font-jp-subset.git)

## 参考ページ
- https://nixeneko.hatenablog.com/entry/2016/02/06/114348
- https://note.com/npaka/n/nb9d4902f8f4d
- https://www.gradio.app/docs/interface

In [ ]:
#画像をフォントに変換するのに必要なパッケージを追加
!apt-get update
!apt-get install fontforge
!apt-get install python3-fontforge
!apt-get install potrace
!apt-get install zbar-tools libcairo2-dev python3-numpy python3-cairosvg python3-pypdf2 python3-pip libffi-dev
!pip3 install qrcode

!git clone https://github.com/nixeneko/handfontgen.git
%cd handfontgen/handfontgen
# 文字範囲のサブセット
!git clone https://github.com/blivesta/font-jp-subset.git
# GUIに必要なパッケージを追加
!pip install gradio


In [ ]:
#@markdown # フォント文字範囲選択
# 文字範囲を指定
#@markdown 英数字
Alphanumeric = True #@param {type:"boolean"}
#@markdown ひらがな
Hiragana = False #@param {type:"boolean"}
#@markdown カタカナ
Katakana = False #@param {type:"boolean"}
#@markdown 記号
Kigou = False #@param {type:"boolean"}
#@markdown 漢字JIS第一水準
KanjiJIS1 = False #@param {type:"boolean"}
#@markdown 全角英数字
Zenkakueisu = False #@param {type:"boolean"}

fontChar = ""

if Alphanumeric == True:
    with open("/content/handfontgen/handfontgen/font-jp-subset/characters/ASCII.txt") as f:
        fontChar+=f.read()
if Hiragana == True:
    with open("/content/handfontgen/handfontgen/font-jp-subset/characters/ひらがな.txt") as f:
        fontChar+=f.read()
if Katakana == True:
    with open("/content/handfontgen/handfontgen/font-jp-subset/characters/カタカナ.txt") as f:
        fontChar+=f.read()
if Kigou == True:
    with open("/content/handfontgen/handfontgen/font-jp-subset/characters/symbol.txt") as f:
        fontChar+=f.read()
if KanjiJIS1 == True:
    with open("/content/handfontgen/handfontgen/font-jp-subset/characters/漢字JIS第1水準.txt") as f:
        fontChar+=f.read()
if Zenkakueisu == True:
    with open("/content/handfontgen/handfontgen/font-jp-subset/characters/全角英数.txt") as f:
        fontChar+=f.read()

In [ ]:
# Google Drive をマウント
from google.colab import drive
drive.mount('/content/drive')
!mkdir /content/drive/MyDrive/fontdir
!mkdir /content/drive/MyDrive/fontdir/svgdir

In [ ]:
import gradio as gr
import numpy as np
import cv2
import subprocess
import scanchars
import sys, os
import glob
import argparse
import slantcorrection
import passzbar
from tilecharbox import Rect
from util import getgrayimage
import passpotrace
import fontgenfromsvg
import unicodedata

outdir = "/content/drive/MyDrive/fontdir/svgdir"
destfile = "/content/drive/MyDrive/fontdir/handfont.otf"
metadata = fontgenfromsvg.FontMetaData(
            fontname="TekitounaTegakiFont",
            family="TekitounaTegakiFont",
            fullname="TekitounaTegakiFont",
            weight="Regular",
            copyrightnotice="",
            fontversion="0.01",
            familyJP="適当な手書きフォント",
            fullnameJP="適当な手書きフォント",
            ascent=860,
            descent=140
            )
currentChar = fontChar[0]
charNum = 0
def detectresol(val):
    x, y = val.strip().split(',')
    xlst = x.split(':')
    ylst = y.split(':')
    return [list(map(int, xlst)), list(map(int, ylst))]
# 画像をsvgに変換する関数
def makeSVG(input_img):
    global outdir
    global destfile
    global metadata
    global currentChar
    global charNum
    name = "{:04X}".format(ord(currentChar))
    # 半角全角判定
    charType = unicodedata.east_asian_width(currentChar)
    if charType in 'FWA':
      resol=detectresol("5:8:5,5:8:5")
    else:
      resol = detectresol("5:4:5,5:8:5")
    charNum+=1
    try:
        currentChar = fontChar[charNum]
    except IndexError:
        gr.Error("最後の文字です。フォント作成ボタンを押してください。")
    grayimg = getgrayimage(input_img)
    ret, binimg = cv2.threshold(grayimg,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    imgheight = 1000
    margintop   = int(imgheight * resol[1][0] / resol[1][1]) #-h[px] * (margint[mm]/h[mm])
    marginbottom= int(imgheight * resol[1][2] / resol[1][1]) #-h[px] * (marginb[mm]/h[mm])
    imgwidth = int(imgheight * resol[0][1] / resol[1][1]) #h[px] * (w[mm]/h[mm])
    marginleft  = int(imgheight * resol[0][0] / resol[1][1]) #-h[px] * (marginl[mm]/h[mm])
    marginright = int(imgheight * resol[0][2] / resol[1][1]) #-h[px] * (marginr[mm]/h[mm])

    # potrace cannot set size in px when converting to SVG.
    # set size in pt and convert to SVG, after that, replace pt with px
    optargs = [ "-W%dpt"%(imgwidth + marginleft + marginright),
                "-H%dpt"%(imgheight + margintop + marginbottom),
                "-L%dpt"%(- marginleft), "-R%dpt"%(- marginright),
                "-T%dpt"%(- margintop), "-B%dpt"%(- marginbottom)]
    bsvg = passpotrace.passpotrace(binimg,optargs)
    bsvg = bsvg.replace(b"pt", b"px") # any exceptions?

    #save function
    if outdir != '' and not os.path.isdir(outdir):
        os.makedirs(outdir)
    scanchars.saveasfile(outdir, name, bsvg)
    return currentChar
# svgからフォントを生成
def makeFont():
    fontgenfromsvg.generatefont(destfile, metadata, outdir)
# Blocksの作成
with gr.Blocks() as demo:
  output = gr.Text(label = "現在の文字",value = currentChar)
  input = gr.Paint(label = "文字入力画面",shape=(200,200))
  next = gr.ClearButton(input, value = "次の文字")
  next.click(fn = makeSVG, inputs = input, outputs = output)
  fontBtn = gr.Button("フォント作成")
  fontBtn.click(fn = makeFont)
# 起動
demo.launch(share=False)